In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
import optuna
from lightgbm import LGBMRegressor
from tqdm import tqdm

In [ ]:
path = '/content/drive/MyDrive/DACON/lg_am/'

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
target_list = ['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']

In [ ]:
for i in tqdm(range(len(target_list))):
    globals()['train_{}'.format(target_list[i])] = train[['X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', f'{target_list[i]}']]
    locals()['train_{}'.format(target_list[i])].drop(columns=['X_48', 'X_47', 'X_23', 'X_04'], inplace=True)

  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 14/14 [00:00<00:00, 74.44it/s]


In [ ]:
X = train_Y_06.loc[:, 'X_01':'X_56']
y = train_Y_06.loc[:, 'Y_06':]

In [ ]:
target = test[X.columns]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    param = {
        'metric': 'rmse', 
         'random_state': 42,
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }


    model = LGBMRegressor(**param)  
    model.fit(x_train, y_train, eval_set = [(x_test, y_test)], early_stopping_rounds = 500, verbose = 2000)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred, squared=False)


    return rmse

In [ ]:
study_catboost = optuna.create_study(direction="minimize")
study_catboost.optimize(objective_cat, n_trials=30)

[I 2022-08-05 01:32:59,883] A new study created in memory with name: no-name-11173b99-9088-4705-b65f-896cba2555a3


Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[768]	valid_0's rmse: 1.62099


[I 2022-08-05 01:33:29,114] Trial 0 finished with value: 1.6209862651472413 and parameters: {'reg_alpha': 1.0510590831209259, 'reg_lambda': 4.697936036300781, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.02, 'max_depth': 100, 'num_leaves': 347, 'min_child_samples': 130, 'min_data_per_groups': 2}. Best is trial 0 with value: 1.6209862651472413.


Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[955]	valid_0's rmse: 1.63391


[I 2022-08-05 01:33:40,374] Trial 1 finished with value: 1.6339109134132421 and parameters: {'reg_alpha': 0.014020676155573439, 'reg_lambda': 7.4822870658278005, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.02, 'max_depth': 100, 'num_leaves': 724, 'min_child_samples': 200, 'min_data_per_groups': 1}. Best is trial 0 with value: 1.6209862651472413.


...

[I 2022-08-05 01:47:39,028] Trial 27 finished with value: 1.6355933601902888 and parameters: {'reg_alpha': 1.7275912795140904, 'reg_lambda': 4.169091722875394, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.014, 'max_depth': 100, 'num_leaves': 355, 'min_child_samples': 105, 'min_data_per_groups': 100}. Best is trial 26 with value: 1.517128552165683.


Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[793]	valid_0's rmse: 1.59369


[I 2022-08-05 01:48:10,277] Trial 28 finished with value: 1.5936932218324709 and parameters: {'reg_alpha': 0.33788447828067686, 'reg_lambda': 0.5081587152173775, 'colsample_bytree': 0.7, 'subsample': 0.5, 'learning_rate': 0.014, 'max_depth': 100, 'num_leaves': 456, 'min_child_samples': 32, 'min_data_per_groups': 88}. Best is trial 26 with value: 1.517128552165683.


Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[1009]	valid_0's rmse: 1.61895


[I 2022-08-05 01:48:26,108] Trial 29 finished with value: 1.61895359723227 and parameters: {'reg_alpha': 0.9584497186499449, 'reg_lambda': 3.3969940912287138, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 100, 'num_leaves': 265, 'min_child_samples': 133, 'min_data_per_groups': 60}. Best is trial 26 with value: 1.517128552165683.


In [ ]:
params=study_catboost.best_params 
params['metric'] = 'rmse'
params['random_state'] = 42
params['n_estimators'] = 20000 

In [ ]:
params

{'colsample_bytree': 1.0,
 'learning_rate': 0.014,
 'max_depth': 100,
 'metric': 'rmse',
 'min_child_samples': 1,
 'min_data_per_groups': 75,
 'n_estimators': 20000,
 'num_leaves': 425,
 'random_state': 42,
 'reg_alpha': 1.669363480078105,
 'reg_lambda': 3.382835149837302,
 'subsample': 1.0}

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = LGBMRegressor(**params)
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 2000)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold NRMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[314]	training's rmse: 0.589415	valid_1's rmse: 1.51713
1 Fold NRMSE = 1.5171285521656832
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[478]	training's rmse: 0.477166	valid_1's rmse: 1.76093
2 Fold NRMSE = 1.760934724524731
Training until validation scores don't improve for 500 rounds.


In [ ]:
np.mean(NRMSES)